# 🔋 Energy Consumption Forecasting - Final Submission
This notebook walks through the complete process from data preparation to modeling and forecasting using XGBoost for customer-level energy consumption in Italy and Spain.

## 📁 1. Data Loading & Preprocessing

In [7]:

import pandas as pd
import numpy as np

# Load Italian dataset
df_it = pd.read_csv("/kaggle/input/datathon2025/historical_metering_data_IT.csv", parse_dates=['DATETIME'])
df_it.set_index("DATETIME", inplace=True)

# Load Spanish dataset
df_es = pd.read_csv("/kaggle/input/spaindata/historical_metering_data_ES.csv")
df_es.rename(columns={df_es.columns[0]: "DATETIME"}, inplace=True)
df_es['DATETIME'] = pd.to_datetime(df_es['DATETIME'])
df_es.set_index('DATETIME', inplace=True)

# Load weather and merge, DATETIME COLUMN IS USED TO MERGE BOTH DATASETS
df_weather = pd.read_excel("/kaggle/input/tempppp/spv_ec00_forecasts_es_it.xlsx", parse_dates=["DATETIME"])
df_weather.set_index("DATETIME", inplace=True)

df_it = df_it.merge(df_weather[["spv", "temp"]], left_index=True, right_index=True, how="left")
df_es = df_es.merge(df_weather[["spv", "temp"]], left_index=True, right_index=True, how="left")

# Fill missing values
df_it.fillna(0, inplace=True)
df_es.fillna(0, inplace=True)


## 🛠️ 2. Feature Engineering

In [8]:
# Add time-based features
for df in [df_it, df_es]:
    df['hour'] = df.index.hour
    df['day'] = df.index.day
    df['month'] = df.index.month

# Holiday dates for ES
holiday_dates = [
    "2022-01-01", "2022-01-06", "2022-04-15", "2022-08-15", "2022-10-12", "2022-11-01", "2022-12-06", "2022-12-08",
    "2023-01-06", "2023-04-06", "2023-04-07", "2023-05-01", "2023-08-15", "2023-10-12", "2023-11-01", "2023-12-06", "2023-12-08", "2023-12-25",
    "2024-01-01", "2024-01-06", "2024-03-28", "2024-03-29", "2024-05-01", "2024-08-15"
]
holiday_dates = pd.to_datetime(holiday_dates).date

# Create holiday indicator for Spain
df_es['holiday'] = pd.Series(df_es.index.date, index=df_es.index).isin(holiday_dates).astype(int)

# Ensure Italy also has a holiday column
if 'holiday' not in df_it.columns:
    df_it['holiday'] = 0
else:
    df_it['holiday'] = df_it['holiday'].astype(int)


#  Feature Engineering (Explored but Not Included in Final Model)

In [18]:
# Although not used in the final model, several features were engineered or tested and found to have potential value:

# 1. Customer Active Status Indicator
# What: A binary flag per timestamp per customer to denote whether a customer was active (i.e., had metered consumption) or inactive (i.e., value was missing).
# Load active status CSV
df_active = pd.read_csv("/kaggle/input/active-statusfile/active_status_wide_IT.csv", parse_dates=["DATETIME"])
df_active.set_index("DATETIME", inplace=True)

# Quick look
df_active.head()




,VALUEMWHMETERINGDATA_customerIT_1,VALUEMWHMETERINGDATA_customerIT_2,VALUEMWHMETERINGDATA_customerIT_3,VALUEMWHMETERINGDATA_customerIT_4,VALUEMWHMETERINGDATA_customerIT_5,VALUEMWHMETERINGDATA_customerIT_6,VALUEMWHMETERINGDATA_customerIT_7,VALUEMWHMETERINGDATA_customerIT_8,VALUEMWHMETERINGDATA_customerIT_13,VALUEMWHMETERINGDATA_customerIT_14,...,VALUEMWHMETERINGDATA_customerIT_2832,VALUEMWHMETERINGDATA_customerIT_2836,VALUEMWHMETERINGDATA_customerIT_2844,VALUEMWHMETERINGDATA_customerIT_2845,VALUEMWHMETERINGDATA_customerIT_2847,VALUEMWHMETERINGDATA_customerIT_2848,VALUEMWHMETERINGDATA_customerIT_2849,VALUEMWHMETERINGDATA_customerIT_2850,VALUEMWHMETERINGDATA_customerIT_2851,VALUEMWHMETERINGDATA_customerIT_2852
DATETIME,,,,,,,,,,,,,,,,,,,,,
2022-01-01 00:00:00,0,0,0,0,0,1,0,1,1,0,...,0,0,0,1,1,1,1,1,1,0
2022-01-01 01:00:00,0,0,0,0,0,1,0,1,1,0,...,0,0,0,1,1,1,1,1,1,0
2022-01-01 02:00:00,0,0,0,0,0,1,0,1,1,0,...,0,0,0,1,1,1,1,1,1,0
2022-01-01 03:00:00,0,0,0,0,0,1,0,1,1,0,...,0,0,0,1,1,1,1,1,1,0
2022-01-01 04:00:00,0,0,0,0,0,1,0,1,1,0,...,0,0,0,1,1,1,1,1,1,0


## 🤖 3. Modeling & Forecasting using XGBoost

In [4]:

import xgboost as xgb
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error

# ⚙️ Settings
LAGS = [1, 24, 48]
FORECAST_HOURS = 24 * 31  # 31 days

# Score weights
SCORING_WEIGHTS_IT = (0.5, 1.0)
SCORING_WEIGHTS_ES = (0.5, 2.5)
def create_lagged_features(series, lags):
    return pd.concat({f'lag_{lag}': series.shift(lag) for lag in lags}, axis=1)

def prepare_features(df, target_col, ext_features):
    data = df[[target_col] + ext_features].copy()
    data[target_col] = data[target_col].fillna(0)

    lagged = create_lagged_features(data[target_col], LAGS)
    full = pd.concat([lagged, data[ext_features], data[target_col]], axis=1).dropna()

    X = full.drop(columns=[target_col])
    y = full[target_col]
    return X, y

def score_portfolio(preds_list, actuals_list, weights):
    pred_flat = np.concatenate(preds_list)
    actual_flat = np.concatenate(actuals_list)
    abs_err = np.sum(np.abs(pred_flat - actual_flat))
    bias_err = np.abs(np.sum(pred_flat - actual_flat))
    return weights[0] * abs_err + weights[1] * bias_err


### 🇮🇹 Forecast for Italy

In [5]:
df = df_it.copy()
customer_cols = [col for col in df.columns if col.startswith("VALUEMWHMETERINGDATA_customerIT_")]
external_features = ['spv', 'temp', 'holiday', 'hour', 'day', 'month']

forecast_results = {}
all_preds, all_actuals = [], []

for cust in tqdm(customer_cols, desc="🔮 Italy Customers"):
    try:
        X, y = prepare_features(df, cust, external_features)
        if len(X) < FORECAST_HOURS + 100: continue

        X_train, X_test = X.iloc[:-FORECAST_HOURS], X.iloc[-FORECAST_HOURS:]
        y_train, y_test = y.iloc[:-FORECAST_HOURS], y.iloc[-FORECAST_HOURS:]

        model = xgb.XGBRegressor(n_estimators=300, learning_rate=0.05, n_jobs=-1)
        model.fit(X_train, y_train)

        preds = model.predict(X_test)
        forecast_results[cust] = {"actual": y_test, "predicted": preds}

        all_preds.append(preds)
        all_actuals.append(y_test.values)

    except Exception as e:
        print(f"⚠️ Skipping {cust}: {e}")

score_it = score_portfolio(all_preds, all_actuals, SCORING_WEIGHTS_IT)
print(f"✅ 🇮🇹 Italy Portfolio Score: {score_it:.4f}")

🔮 Italy Customers: 100%|██████████| 1696/1696 [09:51<00:00,  2.87it/s]

✅ 🇮🇹 Italy Portfolio Score: 14215.7627


### 🇪🇸 Forecast for Spain

In [6]:
df = df_es.copy()
customer_cols = [col for col in df.columns if col.startswith("VALUEMWHMETERINGDATA_customerES_")]
external_features = ['spv', 'temp', 'holiday', 'hour', 'day', 'month']

forecast_results = {}
all_preds, all_actuals = [], []

for cust in tqdm(customer_cols, desc="🔮 Spain Customers"):
    try:
        X, y = prepare_features(df, cust, external_features)
        if len(X) < FORECAST_HOURS + 100: continue

        X_train, X_test = X.iloc[:-FORECAST_HOURS], X.iloc[-FORECAST_HOURS:]
        y_train, y_test = y.iloc[:-FORECAST_HOURS], y.iloc[-FORECAST_HOURS:]

        model = xgb.XGBRegressor(n_estimators=300, learning_rate=0.05, n_jobs=-1)
        model.fit(X_train, y_train)

        preds = model.predict(X_test)
        forecast_results[cust] = {"actual": y_test, "predicted": preds}

        all_preds.append(preds)
        all_actuals.append(y_test.values)

    except Exception as e:
        print(f"⚠️ Skipping {cust}: {e}")

score_es = score_portfolio(all_preds, all_actuals, SCORING_WEIGHTS_ES)
print(f"✅ 🇪🇸 Spain Portfolio Score: {score_es:.4f}")

🔮 Spain Customers: 100%|██████████| 98/98 [00:41<00:00,  2.34it/s]

✅ 🇪🇸 Spain Portfolio Score: 1367.8101


# PREDICTING AUGUST


In [15]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm

# ⚙️ Configuration
LAGS = [1, 24, 48]
external_features = ['spv', 'temp', 'holiday', 'hour', 'day', 'month']

# 📁 Load historical IT data
df_it = pd.read_csv("/kaggle/input/datathon2025/historical_metering_data_IT.csv", parse_dates=["DATETIME"])
df_it.set_index("DATETIME", inplace=True)

# 📁 Load weather file (includes future values)
df_weather = pd.read_excel("/kaggle/input/tempppp/spv_ec00_forecasts_es_it.xlsx", parse_dates=["DATETIME"])
df_weather.set_index("DATETIME", inplace=True)

# 🔁 Merge features
df_it = df_it.merge(df_weather[["spv", "temp"]], how="left", left_index=True, right_index=True)
df_it.fillna(0, inplace=True)

# 🕓 Time features
df_it["hour"] = df_it.index.hour
df_it["day"] = df_it.index.day
df_it["month"] = df_it.index.month
df_it["holiday"] = 0  # Italy has no holidays in this context

# 🔍 Customer columns
customer_cols = [col for col in df_it.columns if col.startswith("VALUEMWHMETERINGDATA_customerIT_")]

# 📅 Prepare future_df (real features for August)
future_df = df_weather.loc["2024-08-01":"2024-08-31 23:00:00", ["spv", "temp"]].copy()
future_df["hour"] = future_df.index.hour
future_df["day"] = future_df.index.day
future_df["month"] = future_df.index.month
future_df["holiday"] = 0  # no holidays in August for Italy

# 🔮 Forecast loop
forecast_august = {}

def create_lagged_features(series, lags):
    return pd.concat({f'lag_{l}': series.shift(l) for l in lags}, axis=1)

for cust in tqdm(customer_cols, desc="🔮 Predicting August 2024"):
    try:
        data = df_it[[cust] + external_features].copy()
        data[cust] = data[cust].fillna(0)

        # Add lag features to training data
        lagged = create_lagged_features(data[cust], LAGS)
        train_df = pd.concat([lagged, data[external_features], data[cust]], axis=1).dropna()

        X_train = train_df.drop(columns=[cust])
        y_train = train_df[cust]

        # Prepare X_future with external features
        X_future = future_df[external_features].copy()

        # 🔧 Add dummy lag columns (0) to match training structure
        for lag in ['lag_1', 'lag_24', 'lag_48']:
            X_future[lag] = 0.0
        X_future = X_future[X_train.columns]  # match exact order

        model = xgb.XGBRegressor(n_estimators=300, learning_rate=0.05, n_jobs=-1)
        model.fit(X_train, y_train)

        preds = model.predict(X_future)
        forecast_august[cust] = preds

    except Exception as e:
        print(f"⚠️ Skipping {cust}: {e}")

🔮 Predicting August 2024:  16%|█▌        | 268/1696 [02:01<10:48,  2.20it/s]


KeyboardInterrupt: 